# Домашнее задание 2

**Тема “Создание признакового пространства”**

Продолжим обработку данных с Твиттера. 

1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.

●	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.

●	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.

●	Исключим стоп-слова с помощью stop_words='english'.

●	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().


2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.

●	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.

●	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.

●	Исключим стоп-слова с помощью stop_words='english'.

●	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().

3. Натренируем gensim.models.Word2Vec модель на наших данных.

●	Тренировать будем на токенизированных твитах combine_df['tweet_token']

●	Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.

●	Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.

4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".


5. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).

Давайте проверим векторное представление любого слова из нашего корпуса, например "food".

6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.

Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.
На выходе должен получиться wordvec_df.shape = (49159, 200).



In [2]:
import pandas as pd
import numpy as np

import nltk

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os

**1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.**

In [3]:
import pickle

with open('combine_df.pkl', 'rb') as f:
    combine_df = pickle.load(f)
    
combine_df.head(3)

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"


In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

count_vectorizer = CountVectorizer(max_df=0.9, max_features=1000, stop_words='english')

In [5]:
tweet_stemmed = combine_df['tweet_stemmed'].apply(lambda x: ' '.join(x))

In [6]:
bow_stemmed_tweet = count_vectorizer.fit_transform(tweet_stemmed)

In [7]:
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(bow_stemmed_tweet.toarray(), columns = feature_names).head(3)

,abl,absolut,accept,account,act,action,activ,actor,actual,ad,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
tweet_lemmatized = combine_df['tweet_lemmatized'].apply(lambda x: ' '.join(x))

In [9]:
bow_lemmatized_tweet = count_vectorizer.fit_transform(tweet_lemmatized)

In [10]:
pd.DataFrame(bow_lemmatized_tweet.toarray(), columns = feature_names).head(3)

,abl,absolut,accept,account,act,action,activ,actor,actual,ad,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.**

In [11]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=1000, stop_words='english')

In [12]:
bow_tfidf_tweet_stemmed = tfidf_vectorizer.fit_transform(tweet_stemmed)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(bow_tfidf_tweet_stemmed.toarray(), columns = tfidf_feature_names).head(3)

,abl,absolut,accept,account,act,action,activ,actor,actual,ad,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
bow_tfidf_tweet_lemmatized = tfidf_vectorizer.fit_transform(tweet_lemmatized)
pd.DataFrame(bow_tfidf_tweet_lemmatized.toarray(), columns = tfidf_feature_names).head(3)

,abl,absolut,accept,account,act,action,activ,actor,actual,ad,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**3. Натренируем gensim.models.Word2Vec модель на наших данных**

In [14]:
data_tweet_token = combine_df['tweet_token'].tolist()

In [1]:
# !pip install gensim

In [15]:
from gensim.models import Word2Vec

model_w2v = Word2Vec(data_tweet_token, size=200, window=5, min_count=2, sg=1, hs=0, negative=10, seed=34)

model_w2v.train(data_tweet_token, total_examples=len(data_tweet_token), epochs=20)

(9140704, 11726520)

**4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".**

In [16]:
model_w2v.most_similar(positive=['dinner'])

[('bihdaydinner', 0.5667390823364258),
 ('bolognese', 0.5537190437316895),
 ('cookout', 0.5508849620819092),
 ('waterloo', 0.5413076281547546),
 ('spaghetti', 0.5365110635757446),
 ('shawarma', 0.5336673259735107),
 ('sissy', 0.5332685708999634),
 ('whoopppp', 0.5309914350509644),
 ('tacotuesday', 0.530888557434082),
 ('burritos', 0.5298112034797668)]

In [17]:
model_w2v.most_similar(positive=['trump'])

[('donald', 0.5422693490982056),
 ('unfit', 0.5365127325057983),
 ('conman', 0.5344905257225037),
 ('chopra', 0.5307890772819519),
 ('dumptrump', 0.5238311886787415),
 ('suppoer', 0.5172950029373169),
 ('trumptrain', 0.5167009234428406),
 ('impeachment', 0.5147131681442261),
 ('donaldtrump', 0.5098357796669006),
 ('trumpleaks', 0.5053690671920776)]

**5. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).**

**Давайте проверим векторное представление любого слова из нашего корпуса, например "food".**

In [18]:
model_w2v['food']

array([-0.2887065 ,  0.1975374 ,  0.10542163,  0.21252064, -0.26215616,
       -0.573694  ,  0.02430337,  0.56806916, -0.48203284,  0.30741012,
       -0.34817532, -0.04092561,  0.11440487, -0.01827279, -0.1316267 ,
        0.37053022,  0.7454078 , -0.78203684,  0.49305218,  0.03588258,
        0.49459723,  0.09766939,  0.2378354 , -0.474187  ,  0.8461422 ,
       -0.23192576, -0.26753476,  0.51622367, -0.34041068,  0.3021215 ,
        0.13277054, -0.48531556, -0.3964209 ,  0.23302357,  0.24161778,
       -0.00154981,  0.60399014,  0.9845133 , -0.0989326 , -0.6499488 ,
       -0.6367183 ,  0.33607194, -0.06404261, -0.23017977,  0.04155174,
       -0.12032308,  0.30071902, -0.20983782, -0.23490082,  0.20417733,
       -0.43347943,  0.05762821, -0.65933514, -1.0076306 , -0.26410097,
        0.8013877 ,  0.10366847,  0.44637024,  0.39680564, -0.09409378,
        0.02863957,  0.58422667,  1.013563  ,  0.29522014,  0.8214965 ,
       -0.00806085,  0.07538228,  0.5204812 ,  0.1957473 ,  0.60

**6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.**

**Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.
На выходе должен получиться wordvec_df.shape = (49159, 200).**

In [22]:
def tweets2vec(tweets_list, size=200):
    n_words = len(tweets_list)
    vec = np.zeros((1, size))
    if n_words == 0:
        return vec
    for word in tweets_list:
        try:
            vec += model_w2v[word].reshape((1, size))
        except KeyError:
            continue
    return vec / n_words

In [23]:
wordvec_df = combine_df['tweet_token'].apply(tweets2vec)
wordvec_df = np.concatenate(wordvec_df)

In [24]:
wordvec_df.shape

(49159, 200)

In [25]:
with open('wordvec_df.pkl', 'wb') as f:
    pickle.dump(wordvec_df, f)